<a href="https://colab.research.google.com/github/N-Ndaba/deli/blob/master/JULY_Virtual_Screening_of_LC_MS_MSC_Siliqua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages

!pip install rdkit

In [ ]:
# Install necessary packages
# Install necessary packages
!pip install chembl_webresource_client tensorflow numpy pandas matplotlib seaborn rdkit keras-tuner scikit-learn
!pip install rdkit
!pip install xgboost
!pip install lightgbm
!pip install pubchempy
!pip install lazypredict
!pip install mordred

## Importing necessary libraries

In [ ]:

!pip install numpy==1.23.5 --force-reinstall


In [ ]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd
from glob import glob

# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Importing necessary modules from the RDKit library
# Importing the Chem module for basic chemistry functions
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem, PandasTools, MACCSkeys, AtomPairs, rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.Chem.rdmolops import PatternFingerprint
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.AtomPairs.Pairs import GetAtomPairFingerprintAsBitVect
pd.set_option('display.max_rows', None)

# Importing various submodules from rdkit.Chem for chemical descriptors and fingerprints
from rdkit.Chem import Descriptors   # For calculating molecular descriptors
from rdkit.Chem import MACCSkeys      # For generating MACCS keys fingerprints
from rdkit.Chem import AllChem          # For various chemical functions, including fingerprint generation
from rdkit.Chem import rdMolDescriptors   # For calculating more advanced molecular descriptors

# Importing the Fingerprinter class from rdkit.Chem.EState for EState fingerprints
from rdkit.Chem.EState import Fingerprinter as EStateFingerprinter

# Library to split data in train and test
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
import xgboost as xgb
import lightgbm as lgb
import pubchempy as pcp
from rdkit.Chem import Crippen

# To get diferent metric scores
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
    precision_recall_curve,
    roc_curve,
    matthews_corrcoef,
    r2_score
)

import warnings
warnings.filterwarnings("ignore")

# Importing the new_client module from the ChEMBL web resource client library
# The ChEMBL web resource client is used to access the ChEMBL database, a large bioactivity database
from chembl_webresource_client.new_client import new_client


from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, roc_auc_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier # Import MLPClassifier


from rdkit.Chem import AllChem, rdFingerprintGenerator


In [ ]:
df = pd.read_csv("/content/LC_MS Analysis_carob_tree_ACS.csv", quotechar='"')
print(df.head())


           SpectrumID  Precursor_MZ  \
0  CCMSLIB00012429248       727.355   
1  CCMSLIB00004707071       809.433   
2  CCMSLIB00004707071       809.433   
3  CCMSLIB00012859861       389.130   
4  CCMSLIB00000839196       225.066   

                                       Compound_Name molecular_formula  \
0                                    NCGC00385704-01         C35H54O13   
1  (1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...         C42H66O15   
2  (1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...         C42H66O15   
3                                           lactitol         C12H24O11   
4                        phenazine-1-carboxylic acid         C13H8N2O2   

                                              Smiles  subclass   Adduct  
0  C[C@@H]1O[C@@H](O[C@H]2CC[C@@]3(C)C(CC[C@@H]4C...       NaN  [M+FA]-  
1  C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...       NaN   [M-H]-  
2  C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...       NaN   [M-H]-  
3  OC[C@@H](O)[C@@H](O[C

In [ ]:
df

,SpectrumID,Precursor_MZ,Compound_Name,molecular_formula,Smiles,subclass,Adduct
0,CCMSLIB00004692374,277.093,"2-methylidene-4-[(2R,3R,4S,5S,6R)-3,4,5-trihyd...",C11H18O8,C=C(CCO[C@H]1[C@@H]([C@H]([C@@H]([C@H](O1)CO)O...,NaN,[M-H]-
1,CCMSLIB00012307407,522.203,Melezitose,C18H32O16,OC[C@H]1O[C@@](CO)(O[C@H]2O[C@H](CO)[C@@H](O)[...,Carbohydrates and carbohydrate conjugates,[M+NH4]+
2,CCMSLIB00012344375,503.162,RAFFINOSE,C18H32O16,OC[C@H]1O[C@H](OC[C@H]2O[C@H](O[C@]3(CO)O[C@H]...,Carbohydrates and carbohydrate conjugates,[M-H]-
3,CCMSLIB00003137162,267.134,"(2S)-2-[[(2S,3R)-2-amino-3-hydroxybutanoyl]ami...",C13H18N2O4,C[C@H]([C@@H](C(=O)N[C@@H](CC1=CC=CC=C1)C(=O)O...,"Amino acids, peptides, and analogues",M+H
4,CCMSLIB00012443865,430.192,6-O-Isobutyryl-alpha-D-glucopyranosyl alpha-D-...,C16H28O12,CC(C)C(=O)OC[C@@H]1[C@H]([C@@H]([C@H]([C@H](O1...,Carbohydrates and carbohydrate conjugates,[M+NH4]+
5,CCMSLIB00011428269,339.131,"11,12-dihydroxy-5Z,8Z,14Z-eicosatrienoic acid;",C20H34O4,C1=CC=C(C=C1)C(C(=O)N)O[C@H]2[C@@H]([C@H]([C@@...,Eicosanoids,M-H
6,CCMSLIB00000849854,520.167,[6-O-(beta-D-Glucopyranosyl)-beta-D-glucopyran...,C20H29NO12,NC(=O)C(O[C@@H]1O[C@H](CO[C@@H]2O[C@H](CO)[C@@...,Fatty acyl glycosides,M+FA-H
7,CCMSLIB00004715754,492.208,"(2R,3S,4S,5R,6R)-2-(hydroxymethyl)-6-[[(2R,3S,...",C21H30O12,C=CCC1=CC(=C(C=C1)O[C@H]2[C@@H]([C@H]([C@@H]([...,Carbohydrates and carbohydrate conjugates,[M+NH4]+
8,CCMSLIB00012867720,387.114,sucrose,C12H22O11,OC[C@H]1O[C@@](CO)(O[C@H]2O[C@H](CO)[C@@H](O)[...,Carbohydrates and carbohydrate conjugates,[M+FA]-
9,CCMSLIB00012414659,299.062,4-N-(2-morpholin-4-ylethyl)-6-thiophen-2-ylpyr...,C14H19N5OS,C1COCCN1CCNC2=NC(=NC(=C2)C3=CC=CS3)N,Pyrimidines and pyrimidine derivatives,[M+H]+


In [ ]:
df = pd.read_csv("/content/LC-MSMS_results_with_gnps.tsv", sep="\t")
print(df.head())


           SpectrumID  #Scan#  SpectrumFile              LibraryName  \
0  CCMSLIB00003139086      40  specs_ms.mgf  GNPS-NIST14-MATCHES.mgf   
1  CCMSLIB00004692374      15  specs_ms.mgf                 MONA.mgf   
2  CCMSLIB00012307407     106  specs_ms.mgf      MSNLIB-POSITIVE.mgf   
3  CCMSLIB00012344375     101  specs_ms.mgf      MSNLIB-NEGATIVE.mgf   
4  CCMSLIB00012353079      68  specs_ms.mgf      MSNLIB-NEGATIVE.mgf   

    MQScore   TIC_Query  RT_Query  MZErrorPPM  SharedPeaks  MassDiff  ...  \
0  0.904710    1000.000         0     3.09763            7  0.001007  ...   
1  0.903847  252000.000         0  6618.66000            4  1.833980  ...   
2  0.880541    2000.000         0     3.85704            6  0.002014  ...   
3  0.826145    2000.000         0     2.00150            5  0.001007  ...   
4  0.793656     999.999         0  2709.75000            4  1.048980  ...   

   molecular_formula                     InChIKey InChIKey-Planar  \
0                NaN               

In [ ]:
df

,SpectrumID,#Scan#,SpectrumFile,LibraryName,MQScore,TIC_Query,RT_Query,MZErrorPPM,SharedPeaks,MassDiff,...,molecular_formula,InChIKey,InChIKey-Planar,superclass,class,subclass,npclassifier_superclass,npclassifier_class,npclassifier_pathway,library_usi
0,CCMSLIB00003139086,40,specs_ms.mgf,GNPS-NIST14-MATCHES.mgf,0.904710,1000.000,0,3.09763,7,0.001007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00003139086
1,CCMSLIB00004692374,15,specs_ms.mgf,MONA.mgf,0.903847,252000.000,0,6618.66000,4,1.833980,...,C11H18O8,QJALTHDWQWOLLT-ZBGLXGBJSA-N,QJALTHDWQWOLLT,NaN,NaN,NaN,Fatty acyl glycosides,Fatty acyl glycosides of mono- and disaccharides,Fatty acids,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00004692374
2,CCMSLIB00012307407,106,specs_ms.mgf,MSNLIB-POSITIVE.mgf,0.880541,2000.000,0,3.85704,6,0.002014,...,C18H32O16,QWIZNVHXZXRPDR-WSCXOGSTSA-N,QWIZNVHXZXRPDR,Organic oxygen compounds,Organooxygen compounds,Carbohydrates and carbohydrate conjugates,Saccharides,Disaccharides|Polysaccharides,Carbohydrates,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00012307407
3,CCMSLIB00012344375,101,specs_ms.mgf,MSNLIB-NEGATIVE.mgf,0.826145,2000.000,0,2.00150,5,0.001007,...,C18H32O16,MUPFEKGTMRGPLJ-ZQSKZDJDSA-N,MUPFEKGTMRGPLJ,Organic oxygen compounds,Organooxygen compounds,Carbohydrates and carbohydrate conjugates,Saccharides,Disaccharides|Polysaccharides,Carbohydrates,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00012344375
4,CCMSLIB00012353079,68,specs_ms.mgf,MSNLIB-NEGATIVE.mgf,0.793656,999.999,0,2709.75000,4,1.048980,...,C12H22O11,RULSWEULPANCDV-PIXUTMIVSA-N,RULSWEULPANCDV,Lipids and lipid-like molecules,Fatty Acyls,Fatty acyl glycosides,Saccharides,Disaccharides,Carbohydrates,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00012353079
5,CCMSLIB00003137162,13,specs_ms.mgf,GNPS-NIST14-MATCHES.mgf,0.765442,999.999,0,3911.94000,4,1.045010,...,C13H18N2O4,NYQBYASWHVRESG-MIMYLULJSA-N,NYQBYASWHVRESG,Organic acids and derivatives,Carboxylic acids and derivatives,"Amino acids, peptides, and analogues",Small peptides,Dipeptides,Amino acids and Peptides,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00003137162
6,CCMSLIB00012443865,83,specs_ms.mgf,MSNLIB-POSITIVE.mgf,0.761991,8999.230,0,2.34100,4,0.001007,...,C16H28O12,QVYMNCVNZVHQEC-HPBJJQKWSA-N,QVYMNCVNZVHQEC,Organic oxygen compounds,Organooxygen compounds,Carbohydrates and carbohydrate conjugates,Monoterpenoids,NaN,Terpenoids,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00012443865
7,CCMSLIB00011428269,49,specs_ms.mgf,NEO-MSMS.mgf,0.722185,3985.040,0,5334.45000,4,1.798980,...,C20H34O4,LRPPQRCHCPFBPE-FYVRRLHDSA-N,LRPPQRCHCPFBPE,Lipids and lipid-like molecules,Fatty Acyls,Eicosanoids,Eicosanoids,Hepoxilins|Hydroxy-hydroperoxyeicosatrienoic a...,Fatty acids,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00011428269
8,CCMSLIB00011428348,45,specs_ms.mgf,NEO-MSMS.mgf,0.713091,156950.000,0,5733.49000,6,1.922000,...,C20H32O4,XYDVGNAQQFWZEF-VXNZHGHESA-N,XYDVGNAQQFWZEF,Lipids and lipid-like molecules,Fatty Acyls,Eicosanoids,Eicosanoids,Hydroxy-hydroperoxyeicosatetraenoic acids,Fatty acids,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00011428348
9,CCMSLIB00000849854,104,specs_ms.mgf,GNPS-NIH-NATURALPRODUCTSLIBRARY_ROUND2_NEGATIV...,0.712824,5000.000,0,1937.83000,8,1.008000,...,C20H29NO12,OCQMNDCGKQJTNY-BVXMNXTGSA-N,OCQMNDCGKQJTNY,Lipids and lipid-like molecules,Fatty Acyls,Fatty acyl glycosides,NaN,NaN,NaN,mzspec:GNPS:GNPS-LIBRARY:CCMSLIB00000849854


In [ ]:
# Save individual dataframes to files Save to CSV
df.to_csv('Lc_MS_carob_tree.csv', index=False)

In [ ]:
missing_values_count = df['Smiles'].isna().sum()
print(f"Number of missing values in 'Smiles': {missing_values_count}")

Number of missing values in 'Smiles': 0


In [ ]:
# Remove rows with missing values in the 'standard_units' column
lcms_ms = df.dropna(subset=['Smiles'])

In [ ]:
lcms_ms = df

In [ ]:
# Save individual dataframes to files Save to CSV
lcms_ms.to_csv('filtered_LC_MS_carob_tree.csv', index=False)

In [ ]:
lcms_ms= pd.read_csv('/content/filtered_LC_MS_carob_tree.csv')

In [ ]:
lcms_ms

,SpectrumID,Precursor_MZ,Compound_Name,molecular_formula,Smiles,subclass,Adduct
0,CCMSLIB00012429248,727.355,NCGC00385704-01,C35H54O13,C[C@@H]1O[C@@H](O[C@H]2CC[C@@]3(C)C(CC[C@@H]4C...,NaN,[M+FA]-
1,CCMSLIB00004707071,809.433,"(1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...",C42H66O15,C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...,NaN,[M-H]-
2,CCMSLIB00004707071,809.433,"(1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...",C42H66O15,C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...,NaN,[M-H]-
3,CCMSLIB00012859861,389.130,lactitol,C12H24O11,OC[C@@H](O)[C@@H](O[C@@H]1O[C@H](CO)[C@H](O)[C...,NaN,[M+FA]-
4,CCMSLIB00000839196,225.066,phenazine-1-carboxylic acid,C13H8N2O2,O=C(O)C1=CC=CC2=NC3=CC=CC=C3N=C12,NaN,M+H
5,CCMSLIB00012859861,389.130,lactitol,C12H24O11,OC[C@@H](O)[C@@H](O[C@@H]1O[C@H](CO)[C@H](O)[C...,NaN,[M+FA]-
6,CCMSLIB00004720685,357.001,Gossypetin,C15H10O8,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2c(O)c(O)cc(O)c12,NaN,[M+K]+
7,CCMSLIB00000849092,442.265,"NCGC00384922-01!(2R,3R,4S,5S,6R)-2-octoxy-6-[[...",C19H36O10,CCCCCCCCO[C@@H]1O[C@H](CO[C@@H]2OC[C@@H](O)[C@...,NaN,M+NH4
8,CCMSLIB00006123224,259.022,fructose 6-phosphate - 30.0 eV,C6H13O9P,C([C@H]([C@H]([C@@H](C(=O)CO)O)O)O)OP(=O)(O)O,NaN,M-H
9,CCMSLIB00012400702,535.203,"NCGC00380141-01![3,4,5-trihydroxy-6-[[3,4,5-tr...",C22H34O12,CC1=C(C(=O)OC2OC(COC3OC(CO)C(O)C(O)C3O)C(O)C(O...,NaN,[M+FA]-


## Virtual Screening

#### Load dataset from LOTUS Database



In [ ]:
pip install python-docx

In [ ]:
import pandas as pd
from docx import Document


# Create Word document
doc = Document()
doc.add_heading('LC-MS/MS Identified Compounds', level=1)

# Add a table
table = doc.add_table(rows=1, cols=len(lcms_ms_data.columns))
table.style = 'Table Grid'

# Add header row
hdr_cells = table.rows[0].cells
for i, column_name in enumerate(lcms_ms_data.columns):
    hdr_cells[i].text = str(column_name)

# Add data rows
for index, row in lcms_ms_data.iterrows():
    row_cells = table.add_row().cells
    for i, item in enumerate(row):
        row_cells[i].text = str(item)

# Save the document
doc.save('lcms_identifiednew_compounds.docx')


In [ ]:
import pandas as pd

#  Select important columns
important_columns = [
    'Smiles','Compound_Name'
]

# Keep only existing columns in the filtered data
existing_columns = [col for col in important_columns if col in lcms_ms.columns]
lcms_ms_data = lcms_ms[existing_columns]

# Display success message and first few rows
print("✅ File loaded, cleaned, and filtered successfully!")
print(lcms_ms_data.head())


✅ File loaded, cleaned, and filtered successfully!
                                              Smiles  \
0  C[C@@H]1O[C@@H](O[C@H]2CC[C@@]3(C)C(CC[C@@H]4C...   
1  C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...   
2  C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...   
3  OC[C@@H](O)[C@@H](O[C@@H]1O[C@H](CO)[C@H](O)[C...   
4                  O=C(O)C1=CC=CC2=NC3=CC=CC=C3N=C12   

                                       Compound_Name  
0                                    NCGC00385704-01  
1  (1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...  
2  (1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...  
3                                           lactitol  
4                        phenazine-1-carboxylic acid  


In [ ]:
lcms_ms_data

,Smiles,Compound_Name
0,C[C@@H]1O[C@@H](O[C@H]2CC[C@@]3(C)C(CC[C@@H]4C...,NCGC00385704-01
1,C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...,"(1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1..."
2,C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...,"(1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1..."
3,OC[C@@H](O)[C@@H](O[C@@H]1O[C@H](CO)[C@H](O)[C...,lactitol
4,O=C(O)C1=CC=CC2=NC3=CC=CC=C3N=C12,phenazine-1-carboxylic acid
5,OC[C@@H](O)[C@@H](O[C@@H]1O[C@H](CO)[C@H](O)[C...,lactitol
6,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2c(O)c(O)cc(O)c12,Gossypetin
7,CCCCCCCCO[C@@H]1O[C@H](CO[C@@H]2OC[C@@H](O)[C@...,"NCGC00384922-01!(2R,3R,4S,5S,6R)-2-octoxy-6-[[..."
8,C([C@H]([C@H]([C@@H](C(=O)CO)O)O)O)OP(=O)(O)O,fructose 6-phosphate - 30.0 eV
9,CC1=C(C(=O)OC2OC(COC3OC(CO)C(O)C(O)C3O)C(O)C(O...,"NCGC00380141-01![3,4,5-trihydroxy-6-[[3,4,5-tr..."


#### Load Stacking Ensemble model and virtual screening

In [ ]:
from rdkit.Chem import MACCSkeys, rdMolDescriptors, RDKFingerprint
import joblib
import pubchempy as pcp

# Load the trained stacked model
stacking_classifier = joblib.load('/content/stacking_classifier (4).pkl')

In [ ]:
# Define fingerprint functions for molecular descriptors

def get_atom_pair_fp(mol):
    """
    Generate the Atom Pair fingerprint for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: Atom Pair fingerprint as a list of integers
    """
    return list(rdFingerprintGenerator.GetAtomPairGenerator().GetCountFingerprint(mol))

def get_maccs_keys(mol):
    """
    Generate the MACCS keys fingerprint for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: MACCS keys fingerprint as a list of integers
    """
    return list(MACCSkeys.GenMACCSKeys(mol))

def get_2d_atom_pair_fp(mol):
    """
    Generate the 2D Atom Pair fingerprint for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: 2D Atom Pair fingerprint as a list of bits
    """
    return list(rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol))

def get_cdk_fp(mol):
    """
    Generate the Topological Torsion fingerprint (CDK) for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: Topological Torsion fingerprint as a list of bits
    """
    return list(rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(mol))

def get_cdk_extended_fp(mol):
    """
    Generate the Extended Morgan fingerprint (CDK) with radius 3 and 2048 bits for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: Extended Morgan fingerprint as a list of bits
    """
    return list(Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048))

def get_cdk_graph_fp(mol):
    """
    Generate the RDKit Graph fingerprint for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: RDKit Graph fingerprint as a list of bits
    """
    return list(Chem.RDKFingerprint(mol))

def get_circle_fp(mol):
    """
    Generate the Morgan fingerprint with radius 2 and 2048 bits for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: Morgan fingerprint with radius 2 as a list of bits
    """
    return list(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))

def get_estate_fp(mol):
    """
    Generate the E-State fingerprint for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: E-State fingerprint as a list of bits
    """
    fp = EStateFingerprinter.FingerprintMol(mol)
    return list(np.concatenate(fp))

def get_hybrid_fp(mol):
    """
    Generate the Hybrid Morgan fingerprint with chirality and radius 2, using 2048 bits.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: Hybrid Morgan fingerprint as a list of bits
    """
    return list(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, useChirality=True, nBits=2048))

def get_klekota_roth_fp(mol):
    """
    Generate the Klekota-Roth fingerprint with 2048 bits for a molecule.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    list: Klekota-Roth fingerprint as a list of bits
    """
    return list(rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol, nBits=2048))

def get_pubchem_fp(mol):
    """
    Generate the PubChem fingerprint for a molecule using the SMILES representation.

    Args:
    mol (rdkit.Chem.Mol): RDKit molecule object

    Returns:
    np.array: PubChem fingerprint as a numpy array of bits
    """
    smiles = Chem.MolToSmiles(mol)
    compound = pcp.get_compounds(smiles, 'smiles')[0]
    fingerprint = compound.cactvs_fingerprint
    return np.array([int(bit) for bit in fingerprint])

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import MACCSkeys, rdMolDescriptors, RDKFingerprint
import joblib
import pubchempy as pcp

# Load the trained stacking classifier
stacking_classifier = joblib.load('/content/stacking_classifier (4).pkl')

# Fingerprint generation functions
def get_maccs_keys(mol):
    return list(MACCSkeys.GenMACCSKeys(mol))

def get_2d_atom_pair_fp(mol):
    return list(rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol))

def get_cdk_fp(mol):
    return list(rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(mol))

def get_cdk_extended_fp(mol):
    return list(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048))

def get_cdk_graph_fp(mol):
    return list(RDKFingerprint(mol))

def get_circle_fp(mol):
    return list(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))

def get_hybrid_fp(mol):
    return list(rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, useChirality=True, nBits=2048))

def get_klekota_roth_fp(mol):
    return list(rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol, nBits=2048))

def get_pubchem_fp(mol):
    try:
        smiles = Chem.MolToSmiles(mol)
        compound = pcp.get_compounds(smiles, 'smiles')[0]
        fingerprint = compound.cactvs_fingerprint
        return np.array([int(bit) for bit in fingerprint])
    except Exception as e:
        print(f"Error fetching PubChem fingerprint: {e}")
        return np.nan

def safe_get_fingerprint(func, mol):
    """Safely compute fingerprint, handle exceptions."""
    try:
        fingerprint = func(mol)
        if fingerprint is None:
            return np.nan
        return fingerprint
    except Exception as e:
        print(f"Error generating fingerprint: {e}")
        return np.nan

# Map of fingerprint names to their generator functions
fingerprint_functions = {
    'MACCS': get_maccs_keys,
    '2D Atom Pair': get_2d_atom_pair_fp,
    'CDK': get_cdk_fp,
    'CDK Extended': get_cdk_extended_fp,
    'CDK Graph': get_cdk_graph_fp,
    'Circle': get_circle_fp,
    'Estate': get_estate_fp,
    'Hybrid': get_hybrid_fp,
    'Klekota-Roth': get_klekota_roth_fp,
    'Pubchem': get_pubchem_fp
}

def generate_fingerprint_dataframes(lcms_ms_data, fingerprint_functions):
    """Generate fingerprints and store them in separate DataFrames."""
    separate_dfs = {}
    for name, func in fingerprint_functions.items():
        print(f"Generating fingerprints for {name}")
        lcms_ms_data[f'{name}_FP'] = lcms_ms_data['mol'].apply(lambda mol: safe_get_fingerprint(func, mol))
        fingerprint_data = lcms_ms_data[f'{name}_FP'].apply(pd.Series)
        separate_dfs[f'{name}_FP'] = fingerprint_data.fillna(0)
    return separate_dfs

# Assume lcms_ms_data is already loaded as a DataFrame with a 'SMILES' column
output_file = 'Virtual_Screening_Results.csv'

try:
    np_atlas_data = lcms_ms_data
    np_atlas_data['mol'] = np_atlas_data['Smiles'].apply(Chem.MolFromSmiles)
    print("Molecules loaded successfully.")
except Exception as e:
    raise ValueError(f"Error processing input data: {e}")

# Generate all fingerprints
separate_fingerprints = generate_fingerprint_dataframes(np_atlas_data, fingerprint_functions)
combined_fingerprint_data = pd.concat(separate_fingerprints.values(), axis=1)

# Check that input features match what model expects
expected_features = stacking_classifier.n_features_in_
if combined_fingerprint_data.shape[1] != expected_features:
    raise ValueError(f"Feature dimension mismatch. Model expects {expected_features}, but input has {combined_fingerprint_data.shape[1]}.")

# Run virtual screening
try:
    predictions = stacking_classifier.predict(combined_fingerprint_data)
    probabilities = stacking_classifier.predict_proba(combined_fingerprint_data)[:, 1]
    np_atlas_data['Prediction'] = predictions
    np_atlas_data['Probability'] = probabilities
    np_atlas_data.to_csv(output_file, index=False)
    print(f"Virtual screening completed. Results saved to {output_file}.")
    print(np_atlas_data[['Smiles', 'Compound_Name', 'Prediction', 'Probability']].head())
except Exception as e:
    raise RuntimeError(f"Error during prediction: {e}")


Molecules loaded successfully.
Generating fingerprints for MACCS
Generating fingerprints for 2D Atom Pair
Generating fingerprints for CDK


[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION WARNING: please use AtomPairGenerator
[15:45:58] DEPRECATION W

Generating fingerprints for CDK Extended
Generating fingerprints for CDK Graph


[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator


Generating fingerprints for Circle
Generating fingerprints for Estate
Generating fingerprints for Hybrid
Generating fingerprints for Klekota-Roth


[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerator
[15:45:58] DEPRECATION WARNING: please use MorganGenerat

Generating fingerprints for Pubchem
Virtual screening completed. Results saved to Virtual_Screening_Results.csv.
                                              Smiles  \
0  C[C@@H]1O[C@@H](O[C@H]2CC[C@@]3(C)C(CC[C@@H]4C...   
1  C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...   
2  C[C@H]1[C@H](C)CC[C@]2(C(=O)O[C@@H]3O[C@H](CO)...   
3  OC[C@@H](O)[C@@H](O[C@@H]1O[C@H](CO)[C@H](O)[C...   
4                  O=C(O)C1=CC=CC2=NC3=CC=CC=C3N=C12   

                                       Compound_Name  Prediction  Probability  
0                                    NCGC00385704-01           1     0.513218  
1  (1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...           0     0.490138  
2  (1S,2R,4aS,6aR,6bR,10S,12aR,14bS)-1,2,6b,9,9,1...           0     0.490138  
3                                           lactitol           0     0.255751  
4                        phenazine-1-carboxylic acid           0     0.214318  
